In [3]:
%load_ext autoreload
%autoreload 2

from __future__ import division
import numpy as np
import scipy.signal as sg
from scipy.fftpack import rfft, fftfreq
from scipy.io import wavfile
import matplotlib.pyplot as plt
import matplotlib.patches as patches
%matplotlib inline
import matplotlib.colors as colors
from pydub import AudioSegment
import math as m
import tensorflow as tf
import utilities as util

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [10]:
sample_files = []
location = '../samples/'
instruments = ['Trombone', 'Guitar']
for name in instruments:
    for note in (util.note_names('C1', 'C2')):
        for j in range(1, 6):
            file_name = location +name+"/"+name +"_"+note[:-1]+"("+str(j)+").wav"
            sample_files.append(file_name)

In [11]:
sample_files

['../samples/Trombone/Trombone_C(1).wav',
 '../samples/Trombone/Trombone_C(2).wav',
 '../samples/Trombone/Trombone_C(3).wav',
 '../samples/Trombone/Trombone_C(4).wav',
 '../samples/Trombone/Trombone_C(5).wav',
 '../samples/Trombone/Trombone_C#(1).wav',
 '../samples/Trombone/Trombone_C#(2).wav',
 '../samples/Trombone/Trombone_C#(3).wav',
 '../samples/Trombone/Trombone_C#(4).wav',
 '../samples/Trombone/Trombone_C#(5).wav',
 '../samples/Trombone/Trombone_D(1).wav',
 '../samples/Trombone/Trombone_D(2).wav',
 '../samples/Trombone/Trombone_D(3).wav',
 '../samples/Trombone/Trombone_D(4).wav',
 '../samples/Trombone/Trombone_D(5).wav',
 '../samples/Trombone/Trombone_D#(1).wav',
 '../samples/Trombone/Trombone_D#(2).wav',
 '../samples/Trombone/Trombone_D#(3).wav',
 '../samples/Trombone/Trombone_D#(4).wav',
 '../samples/Trombone/Trombone_D#(5).wav',
 '../samples/Trombone/Trombone_E(1).wav',
 '../samples/Trombone/Trombone_E(2).wav',
 '../samples/Trombone/Trombone_E(3).wav',
 '../samples/Trombone/Tr

In [13]:
def tf_normal_peak(vals, dfreq):
    if vals.shape[0] == 1: return vals
    sums = tf.reduce_sum(vals, 0, keepdims=True)
    weights = vals/sums
    freqs = tf.range(tf.cast(tf.shape(vals)[0], 'float64'), dtype='float64')
    mean = tf.reduce_sum(
        tf.cast(weights, 'float64') * tf.cast(freqs[:,None], 'float64'),  
        axis=0,
        keepdims=True)
    
    var = tf.truediv(tf.reduce_sum(tf.cast(((freqs[:,None]-mean)**2), 'float64')*tf.cast(weights, 'float64'), 0),dfreq)
    denom_var = tf.multiply(tf.constant(2.0, dtype='float64'), m.pi)
    denom_var_mult = tf.scalar_mul(denom_var, var)
    denom = tf.sqrt(tf.abs(denom_var_mult))
    dst_peak = tf.truediv(tf.constant(1.0, dtype='float64'),denom)
    peak = (tf.cast(dst_peak, 'float64') * tf.cast(dfreq, 'float64') * tf.cast(tf.squeeze(sums), 'float64')) if (dst_peak != 'Nan') else 0
    return peak

In [14]:
nps_uniq, dfreqs, notes = util.precompute_spect()

smallest_nps = 44100/24 * 4/3 
smallest_nps = 2**int(np.log2(smallest_nps))
full_spec = []
tf_normal_peaks = []

In [19]:
def tf_spect(wav, fs): 
    for nperseg in nps_uniq:
        print('...')
        signals = tf.placeholder(tf.float32)

        frames = tf.contrib.signal.frame(signals, frame_length=np.int32(nperseg),
                                         frame_step=np.int32(nperseg-smallest_nps))
        stfts = tf.contrib.signal.stft(signals, frame_length=np.int32(nperseg),
                                       frame_step=np.int32(nperseg-smallest_nps),
                                       fft_length=np.int32(nperseg))
        #   power_spectrograms = tf.real(stfts * tf.conj(stfts))
        magnitude_spectrograms = tf.abs(stfts)
        full_spec.append(magnitude_spectrograms)

    for nps_ind, slc in notes:
        tf_spec = full_spec[nps_ind]
        tf_np = tf_normal_peak(tf_spec[:,slc], dfreqs[nps_ind])
        full_spec[i,:len(time)] = tf_np


In [ ]:
def tf_gather_data(filename):
    fs, wav = wavfile.read(filename)
    wav = wav.astype(np.double)
    freq, time, spectrogram = tf_spect(wav, fs)
    arg_sort = np.argsort(spectrogram,  axis=0)
    return fs, wav, freq, time, spectrogram, arg_sort

In [17]:
FSS = WAVV = FREQQ = TIMEE = SPECTT = ARG_SORTSS = []
for sample in sample_files:
    fs, wav, freq, time, spectrogram, arg_sort = util.gather_data(sample)
    FSS.append(fs)
    WAVV.append(wav)
    FREQQ.append(freq)
    TIMEE.append(time)
    SPECTT.append(spectrogram)
    ARG_SORTSS.append(arg_sort)

/Users/Peeke/Honors/refactored-robot/Notebooks/utilities.py:116: RuntimeWarning: invalid value encountered in true_divide
  weights = vals/sums # the weights for each time
/Users/Peeke/Honors/.env/lib/python3.6/site-packages/scipy/io/wavfile.py:273: WavFileWarning: Chunk (non-data) not understood, skipping it.
  WavFileWarning)
/Users/Peeke/Honors/.env/lib/python3.6/site-packages/scipy/signal/spectral.py:1773: UserWarning: nperseg = 2048 is greater than input length  = 2, using nperseg = 2
  .format(nperseg, input_length))
/Users/Peeke/Honors/.env/lib/python3.6/site-packages/scipy/signal/spectral.py:1773: UserWarning: nperseg = 4096 is greater than input length  = 2, using nperseg = 2
  .format(nperseg, input_length))


ValueError: noverlap must be less than nperseg.